<think>
Tôi cần phát triển một cơ chế cache để lưu trữ kết quả của 4 loại học tập (Learning Meaning, Learning Card, Learning Flexible, Learning QNA) và sử dụng lại khi người dùng chuyển đổi giữa các tab nếu gen question không thay đổi.

Cách tiếp cận:
1. Tạo một lớp LearningCache để lưu trữ và quản lý dữ liệu cache
2. Thêm logic kiểm tra và sử dụng cache trong các hàm generate
3. Thêm cơ chế xóa cache khi có thay đổi dữ liệu đầu vào

Tôi sẽ sử dụng localStorage để lưu trữ cache này để dữ liệu vẫn tồn tại ngay cả khi người dùng làm mới trang.
</think>

# Thêm cơ chế Cache giữa các ô Learning

Tôi sẽ tạo một hệ thống cache để lưu trữ kết quả của bốn loại learning (Meaning, Card, Flexible, QNA). Điều này sẽ cải thiện hiệu suất khi người dùng chuyển đổi giữa các tab.

## 1. Triển khai lớp LearningCache

Đầu tiên, sửa file `cache.js` đã bị comment:

```javascript:1_srcContentGeneration/src/frontend/src/js/modules/cache.js
class LearningCache {
    constructor() {
        this.CACHE_PREFIX = 'learning_cache_';
        this.modules = ['qna', 'card', 'meaning', 'flexible'];
    }

    clearAll() {
        console.log('Starting cache clear operation...');
        
        // Clear localStorage cache
        this.modules.forEach(module => {
            const key = this.CACHE_PREFIX + module;
            if (localStorage.getItem(key)) {
                localStorage.removeItem(key);
                console.log(`Cleared cache for ${module}`);
            }
        });

        // Verify all caches are cleared
        const remainingCaches = this.modules.filter(module => this.has(module));
        if (remainingCaches.length === 0) {
            console.log('All caches successfully cleared');
        } else {
            console.warn('Some caches remain:', remainingCaches);
        }
    }

    set(module, data, lesson_id) {
        if (!this.modules.includes(module)) {
            console.warn(`Invalid module: ${module}`);
            return;
        }
        
        // Thêm thông tin thời gian cache và lesson_id
        const cacheData = {
            data: data,
            timestamp: Date.now(),
            lesson_id: lesson_id
        };
        
        const key = this.CACHE_PREFIX + module;
        localStorage.setItem(key, JSON.stringify(cacheData));
        console.log(`Cache set for ${module} with lesson_id: ${lesson_id}`);
    }

    get(module, lesson_id) {
        if (!this.modules.includes(module)) {
            console.warn(`Invalid module: ${module}`);
            return null;
        }
        
        const key = this.CACHE_PREFIX + module;
        const cachedString = localStorage.getItem(key);
        
        if (!cachedString) {
            console.log(`No cache found for ${module}`);
            return null;
        }
        
        try {
            const cachedData = JSON.parse(cachedString);
            
            // Kiểm tra nếu cache thuộc về lesson_id hiện tại
            if (cachedData.lesson_id !== lesson_id) {
                console.log(`Cache for ${module} belongs to different lesson_id, ignoring`);
                return null;
            }
            
            console.log(`Cache found for ${module}, cached at: ${new Date(cachedData.timestamp).toLocaleTimeString()}`);
            return cachedData.data;
        } catch (e) {
            console.error(`Error parsing cache for ${module}:`, e);
            return null;
        }
    }

    has(module) {
        if (!this.modules.includes(module)) {
            console.warn(`Invalid module: ${module}`);
            return false;
        }
        const key = this.CACHE_PREFIX + module;
        return localStorage.getItem(key) !== null;
    }
    
    // Xóa cache khi lesson_id thay đổi
    invalidateForNewLesson() {
        this.clearAll();
        console.log('Cache invalidated for new lesson');
    }
}

const learningCache = new LearningCache();
export default learningCache;
```

## 2. Cập nhật các module learning để sử dụng cache

### 2.1 Cập nhật `learningMeaning.js`

```javascript:1_srcContentGeneration/src/frontend/src/js/modules/learningMeaning.js
import { config } from '../config.js';
import { showLoadingDialog, hideLoadingDialog, updateLoadingProgress } from '../utils.js';
import TableLearningMeaningTracking from '../trackings/tableLearningMeaningTracking.js';
import { storagedLessons, generateUniqueId } from '../generateQuestion.js';
import learningCache from './cache.js';

const API_URL = config.production.apiUrl;
let learningMeaningLessons = [];
let rawApiResponse;
let currentLessonId = null;

async function generateLearningMeaning(lessons) {
    try {
        // Set lesson_id from input or generate new one
        currentLessonId = lessons[0]?.lesson_id || generateUniqueId();
        
        // Check cache first
        const cachedData = learningCache.get('meaning', currentLessonId);
        if (cachedData) {
            console.log('Using cached Learning Meaning data');
            learningMeaningLessons = cachedData;
            rawApiResponse = cachedData; // We also cache the raw response
            displayLearningMeaningResults(learningMeaningLessons);
            return;
        }
        
        console.log('No cache found, generating new Learning Meaning data');
        showLoadingDialog();
        updateLoadingProgress(10);
        
        const response = await fetch(`${API_URL}/generate-learning-meaning`, {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ lessons })
        });
        
        updateLoadingProgress(70);
        
        if (!response.ok) {
            const errorData = await response.json();
            throw new Error(errorData.error || `HTTP error! status: ${response.status}`);
        }
        
        const data = await response.json();
        updateLoadingProgress(90);
        
        rawApiResponse = data;
        learningMeaningLessons = data;
        
        // Store in cache
        learningCache.set('meaning', data, currentLessonId);
        
        displayLearningMeaningResults(learningMeaningLessons);
        updateLoadingProgress(100);
    } catch (error) {
        console.error('Error:', error);
        alert(error.message);
    } finally {
        hideLoadingDialog();
    }
}

// Các hàm khác giữ nguyên
// ...

export { learningMeaningLessons, generateLearningMeaning };
```

### 2.2 Cập nhật `learningCard.js`

```javascript:1_srcContentGeneration/src/frontend/src/js/modules/learningCard.js
import { config } from '../config.js';
import { showLoadingDialog, hideLoadingDialog, updateLoadingProgress } from '../utils.js';
import TableLearningCardTracking from '../trackings/tableLearningCardTracking.js';
import { generateUniqueId } from '../generateQuestion.js';
import learningCache from './cache.js';

const API_URL = config.production.apiUrl;
let learningCardLessons = [];
let rawApiResponse;
let currentLessonId = null;

async function generateLearningCard(lessons) {
    try {
        console.log('Generating learning cards with edited lessons:', lessons);

        if (!lessons || !Array.isArray(lessons) || lessons.length === 0) {
            throw new Error('Invalid lessons data');
        }

        currentLessonId = lessons[0]?.lesson_id || generateUniqueId();
        
        // Check cache first
        const cachedData = learningCache.get('card', currentLessonId);
        if (cachedData) {
            console.log('Using cached Learning Card data');
            rawApiResponse = cachedData;
            learningCardLessons = cachedData.map(item => ({
                ...item,
                lesson_id: currentLessonId
            }));
            displayLearningCardResults(learningCardLessons);
            return;
        }
        
        console.log('No cache found, generating new Learning Card data');
        showLoadingDialog();
        updateLoadingProgress(10);
        
        const response = await fetch(`${API_URL}/generate-learning-card`, {
            method: 'POST',
            headers: { 'Content-Type': 'application/json' },
            body: JSON.stringify({ lessons: lessons })
        });
        
        updateLoadingProgress(70);
        
        if (!response.ok) {
            const errorData = await response.json();
            throw new Error(errorData.error || `HTTP error! status: ${response.status}`);
        }
        
        const data = await response.json();
        updateLoadingProgress(90);
        
        if (!data || !Array.isArray(data)) {
            throw new Error('Invalid response format from server');
        }

        rawApiResponse = data;
        learningCardLessons = data.map(item => ({
            ...item,
            lesson_id: currentLessonId
        }));
        
        // Store in cache
        learningCache.set('card', data, currentLessonId);
        
        displayLearningCardResults(learningCardLessons);
        updateLoadingProgress(100);
    } catch (error) {
        console.error('Error in generateLearningCard:', error);
        alert(`Error generating learning cards: ${error.message}`);
    } finally {
        hideLoadingDialog();
    }
}

// Các hàm khác giữ nguyên
// ...

export { learningCardLessons, generateLearningCard };
```

### 2.3 Cập nhật `learningFlexible.js` và `learningQNA.js` 

Thực hiện tương tự với hai module còn lại.

## 3. Cập nhật `generateQuestion.js` để xóa cache khi tạo câu hỏi mới

```javascript:1_srcContentGeneration/src/frontend/src/js/generateQuestion.js
// Thêm import
import learningCache from './modules/cache.js';

// Trong hàm handleGenerateClick(), thêm vào đầu hàm:
async function handleGenerateClick() {
    // Xóa cache khi tạo câu hỏi mới
    learningCache.invalidateForNewLesson();
    
    const activeTab = document.querySelector('.tab-content.active');
    const isStandardForm = activeTab.id === 'standard-form';
    
    // phần còn lại giữ nguyên
    // ...
}
```

## 4. Cập nhật `scripts.js` để tận dụng cache khi chuyển đổi tab

```javascript:1_srcContentGeneration/src/frontend/src/js/scripts.js
// Thêm import cache
import learningCache from './modules/cache.js';

// Trong sự kiện click tab, thêm logic check cache:
document.querySelectorAll('.tab-btn').forEach(button => {
    button.addEventListener('click', function() {
        const tabId = this.dataset.tab;
        
        // Active tab trong UI
        document.querySelectorAll('.tab-btn').forEach(btn => btn.classList.remove('active'));
        this.classList.add('active');
        
        document.querySelectorAll('.tab-content').forEach(content => content.classList.remove('active'));
        document.getElementById(tabId).classList.add('active');

        // Kiểm tra nếu đã có dữ liệu từ generateQuestion
        if (storagedLessons && storagedLessons.length > 0) {
            // Call the appropriate function based on the tab clicked
            if (tabId === 'learning-meaning') {
                generateLearningMeaning(storagedLessons); // Cache sẽ được kiểm tra trong hàm này
            } else if (tabId === 'learning-card') {
                generateLearningCard(storagedLessons); // Cache sẽ được kiểm tra trong hàm này
            } else if (tabId === 'flexible-phrase') {
                generateLearningFlexible(storagedLessons); // Cache sẽ được kiểm tra trong hàm này
            } else if (tabId === 'learning-qna') {
                generateLearningQNA(storagedLessons); // Cache sẽ được kiểm tra trong hàm này
            }
        }
    });
});
```

## Lợi ích của cơ chế cache này:

1. **Giảm thời gian chờ**: Người dùng không cần đợi tạo lại dữ liệu khi chuyển tab.

2. **Tiết kiệm tài nguyên**: Giảm số lượng yêu cầu API đến backend.

3. **Trải nghiệm người dùng tốt hơn**: Chuyển đổi tab mượt mà, nhanh chóng.

4. **Thông minh xử lý lesson_id**: Cache duy trì mối quan hệ với lesson_id, đảm bảo không sử dụng cache từ một bài học trước đó.

5. **Tự động xóa cache** khi người dùng tạo bài học mới.

Với các thay đổi này, người dùng sẽ thấy hiệu suất cải thiện đáng kể khi làm việc với ứng dụng, đặc biệt khi chuyển đổi giữa các tab để xem các loại nội dung khác nhau mà không cần phải tạo lại dữ liệu.
